# Temporal Vectorization

We will talk about the **Mandelbrot set**, it is the set of complex numbers $c$ for which the function $f_c(z)=z^2+c$ does not diverge when iterated from $z=0$, i.e., for which the sequence $f_c(0)$, $f_c(f_c(0))$, etc., remains bounded in **absolute value**. It is very *easy to compute*, but it can take a *very long time* because you need to ensure a given number does not diverge. This is generally done by iterating the computation up to a **maximum number of iterations**, after which, if the number is still within some bounds, it is considered **non-divergent**. Of course, the **more iterations** you do, the **more precision** you get. Notice (and appreciate maybe) the self-repeating patterns below!

![](https://www.labri.fr/perso/nrougier/from-python-to-numpy/data/Fractal-Broccoli-cropped.jpg)

## Helper Function(s) (Can Safely skip!)

In [1]:
def timeit(stmt, globals):
    '''
    Parameters
    ----------
    stmt - function to run
    globals - dictionary of current global variables

    Returns
    -------
    Nothing! Just prints relevant information about run-times.
    '''
    import timeit as _timeit
    import numpy as np

    # Rough approximation of a single run
    trial = _timeit.timeit(stmt, globals=globals, number=1)

    # Maximum duration
    duration = 1.0

    # Number of repeat
    repeat = 3

    # Compute rounded number of trials
    number = max(1,int(10**np.floor(np.log(duration/trial/repeat)/np.log(10))))

    # Only report best run
    best = min(_timeit.repeat(stmt, globals=globals, number=number, repeat=repeat))

    units = {"usec": 1, "msec": 1e3, "sec": 1e6}
    precision = 3
    usec = best * 1e6 / number
    if usec < 1000:
        print("%d loops, best of %d: %.*g usec per loop" % (number, repeat,
                                                            precision, usec))
    else:
        msec = usec / 1000
        if msec < 1000:
            print("%d loops, best of %d: %.*g msec per loop" % (number, repeat,
                                                                precision, msec))
        else:
            sec = msec / 1000
            print("%d loops, best of %d: %.*g sec per loop" % (number, repeat,
                                                               precision, sec))

## Pure Python implementation

In [3]:
def mandelbrot_python(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon=2.0):
  '''
  Parameters
  ----------
  xmin - minimum Re(Z) you want to consider/search over
  xmax - maximum Re(Z) you want to consider/search over
  ymin - minimum Im(Z) you want to consider/search over
  ymax - maximum Im(Z) you want to consider/search over
  xn - number of Re(Z) you want to allow
  yn - number of Im(Z) you want to allow
  maxiter & horizon - heuristics which define maximum iterations to do and upto which point is a number considered non-divergent respectively

  Returns
  -------
  a list numbers, iterations taken before the number diverged / maxiter IF the number did not diverge.
  '''
  def mandelbrot(z, maxiter):
    '''
    Parameters
    ----------
    z - Complex Number
    maxiter - Maximum iterations to check for

    Returns
    -------
    iterations taken before the number diverged / 0 IF the number did not diverge (because each number has to take at-least one step).
    '''
    c = z
    for n in range(maxiter):
        if abs(z) > horizon:
            return n
        z = z*z + c
    return 0
  r1 = [xmin+i*(xmax-xmin)/xn for i in range(xn)]
  r2 = [ymin+i*(ymax-ymin)/yn for i in range(yn)]
  return [mandelbrot(complex(r, i),maxiter) for r in r1 for i in r2]

The main function in this code `mandelbrot(z, complex)` is the slowest! But it is what is involved in computing $f_c(f_c(z))$, The vectorization of such code is not easy! (Has an internal return, which can cause early stopping for a few numbers and not so for other few!)

## Numpy Implementation

In [8]:
import numpy as np

def mandelbrot_numpy(xmin, xmax, ymin, ymax, xn, yn, maxiter, horizon=2.0):
    '''
    Parameters
    ----------
    xmin - minimum Re(Z) you want to consider/search over
    xmax - maximum Re(Z) you want to consider/search over
    ymin - minimum Im(Z) you want to consider/search over
    ymax - maximum Im(Z) you want to consider/search over
    xn - number of Re(Z) you want to allow
    yn - number of Im(Z) you want to allow
    maxiter & horizon - heuristics which define maximum iterations to do and upto which point is a number considered non-divergent respectively

    Returns
    -------
    a tuple x, such that, x[0][i] = final ith complex number before diverging/after maxiter iterations and x[1][i] = iter number of divergence
    or 0 for no divergence
    '''
    X = np.linspace(xmin, xmax, xn, dtype=np.float32)
    Y = np.linspace(ymin, ymax, yn, dtype=np.float32)
    C = X + Y[:,None]*1j #an extra dimension is added using [:, None], just to make it a matrix instead of a vector
    N = np.zeros(C.shape, dtype=int)
    Z = np.zeros(C.shape, np.complex64)
    for n in range(maxiter):
        I = abs(Z) < horizon
        N[I] = n
        Z[I] = Z[I]**2 + C[I]
    I = np.less(abs(Z), horizon)
    N[I] = maxiter
    return Z, N

## Compare

In [9]:
xmin, xmax, xn = -2.25, +0.75, int(3000/3)
ymin, ymax, yn = -1.25, +1.25, int(2500/3)
maxiter = 200
#Ideally you should observe a 5x speedup.
timeit("mandelbrot_python(xmin, xmax, ymin, ymax, xn, yn, maxiter)", globals())
timeit("mandelbrot_numpy(xmin, xmax, ymin, ymax, xn, yn, maxiter)", globals())

1 loops, best of 3: 3.86 sec per loop
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(833, 1000)
(8

Can we go faster? Notice that at each iteration we are performing xn$\times$yn tests, even though we know for sure that there is going to be no update to already diverged numbers! (the solution is a typical space-time complexity trade-off)

## Mandelbrot Fastest

In [12]:
def mandelbrot_numpy_2(xmin, xmax, ymin, ymax, xn, yn, itermax, horizon=2.0):
    '''
    Parameters
    ----------
    xmin - minimum Re(Z) you want to consider/search over
    xmax - maximum Re(Z) you want to consider/search over
    ymin - minimum Im(Z) you want to consider/search over
    ymax - maximum Im(Z) you want to consider/search over
    xn - number of Re(Z) you want to allow
    yn - number of Im(Z) you want to allow
    maxiter & horizon - heuristics which define maximum iterations to do and upto which point is a number considered non-divergent respectively

    Returns
    -------
    a tuple x, such that, x[0][i] = final ith complex number before diverging/after maxiter iterations and x[1][i] = iter number of divergence
    or 0 for no divergence
    '''

    Xi, Yi = np.mgrid[0:xn, 0:yn] # Check out what mgrid does here -> https://numpy.org/doc/stable/reference/generated/numpy.mgrid.html
    Xi, Yi = Xi.astype(np.uint32), Yi.astype(np.uint32)
    X = np.linspace(xmin, xmax, xn, dtype=np.float32)[Xi]
    Y = np.linspace(ymin, ymax, yn, dtype=np.float32)[Yi]
    C = X + Y*1j
    N_ = np.zeros(C.shape, dtype=np.uint32)
    Z_ = np.zeros(C.shape, dtype=np.complex64)
    Xi.shape = Yi.shape = C.shape = xn*yn

    Z = np.zeros(C.shape, np.complex64)
    for i in range(itermax):
        if not len(Z): break

        # Compute for relevant points only
        np.multiply(Z, Z, Z)
        np.add(Z, C, Z)

        # Failed convergence
        I = abs(Z) > horizon
        N_[Xi[I], Yi[I]] = i+1
        Z_[Xi[I], Yi[I]] = Z[I]

        # Keep going with those who have not diverged yet
        I = ~I
        Z = Z[I]
        Xi, Yi = Xi[I], Yi[I]
        C = C[I]
    return Z_.T, N_.T

## Test!

In [13]:
#Ideally you should observe a 10x speedup.
timeit("mandelbrot_numpy_2(xmin, xmax, ymin, ymax, xn, yn, maxiter)", globals())

1 loops, best of 3: 466 msec per loop


As a further exercise, try plotting the set of Mandelbrot Numbers you get, using matplotlib. It's a beautiful pattern.

![Mandelbrot set](https://www.labri.fr/perso/nrougier/from-python-to-numpy/data/mandelbrot.png)

## Thank you. <br>
Code for this notebook is borrowed and edited from [here](https://www.labri.fr/perso/nrougier/from-python-to-numpy/). Feel free to check the book out to explore vectorization in different contexts and examples. This small tutorial was made by [Karan Bania](https://karannb.github.io).<br>
Copyright (2017) Nicolas P. Rougier - BSD license